In [ ]:
!nvidia-smi

Thu Dec  7 13:13:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.3 MB/s eta 0:00:00


In [ ]:
!pip install pandas
!pip install nltk
!pip install gensim
!pip install sentence-transformers

!pip install torch


In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install SentencePiece
!pip install evaluate
!pip install bert_score

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os
import pandas as pd
# import torch
# import torch.nn as nn
# from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# from transformers import LlamaTokenizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
jobs = pd.read_csv("drive/MyDrive/IR_Project/cleaned_data/jobs_cleaned.csv")
jobs

,Unnamed: 0,open_job-href,job_title_simple,company_name,address,salary_lb,salary_hb,job_desc
0,0,https://www.mycareersfuture.gov.sg/job/admin/a...,Administrative Assistant,LYONDELL SOUTH ASIA PTE LTD,78 SHENTON WAY 079120,4400,4800,Report discriminatory job ad to TAFEPRoles & R...
1,1,https://www.mycareersfuture.gov.sg/job/custome...,Customer Service Officer,RECRUITPEDIA PTE. LTD.,NaN,2200,2300,Report discriminatory job ad to TAFEPRoles & R...
2,2,https://www.mycareersfuture.gov.sg/job/admin/j...,JEWELLERY ENGRAVER,ABIRAAME POPULAR JEWELLERS(S) PTE. LTD.,85 SERANGOON ROAD 217990,7200,8500,Report discriminatory job ad to TAFEPRoles & R...
3,3,https://www.mycareersfuture.gov.sg/job/informa...,Junior Production Support Engineer,SEARCH ALLY PTE. LTD.,"CT HUB, 2 KALLANG AVENUE 339407",5000,7000,Report discriminatory job ad to TAFEPRoles & R...
4,4,https://www.mycareersfuture.gov.sg/job/general...,LORRY DRIVER,GLOBAL CONNECTION MANPOWER PTE. LTD.,"TAMPINES INDUSTRIAL PARK A, 9005 TAMPINES STRE...",3200,4700,Report discriminatory job ad to TAFEPRoles & R...
...,...,...,...,...,...,...,...,...
10941,10941,https://www.mycareersfuture.gov.sg/job/repair-...,Assistant Service Engineer #65054,ANRADUS PTE. LTD.,NaN,2500,3500,Roles & ResponsibilitiesJob Description\n\n I...
10942,10942,https://www.mycareersfuture.gov.sg/job/custome...,Customer Service [ Air & Sea freight / Ubi ...,THE SUPREME HR ADVISORY PTE. LTD.,"SHENTON HOUSE, 3 SHENTON WAY 068805",2500,2500,"Roles & ResponsibilitiesSalary: SGD 2,500 + AW..."
10943,10943,https://www.mycareersfuture.gov.sg/job/educati...,Full - Time English Teacher,BOUTIQUE EDUCATION PTE. LTD.,NaN,3200,4200,Roles & ResponsibilitiesTeaching with Boutique...
10944,10944,https://www.mycareersfuture.gov.sg/job/events/...,Events Management Coordinator,ELITEZ & ASSOCIATES PTE. LTD.,"CT HUB, 2 KALLANG AVENUE 339407",2600,3000,Roles & ResponsibilitiesJob Responsibilities:\...


In [ ]:
import json
json_file_path = "drive/MyDrive/IR_Project/vector_doc_map.json"
# Read the dictionary from the JSON file
with open(json_file_path, 'r') as json_file:
    loaded_vector_doc_map = json.load(json_file)

len(loaded_vector_doc_map)

10559

In [ ]:
vec_doc_map_import = {}

for key, val in loaded_vector_doc_map.items() :
  k = tuple(key)
  vec_doc_map_import[k] = val

# vec_doc_map_import.keys()

NameError: ignored

In [ ]:
class Document:
  def __init__(self, row):
    self.details = row

  @property
  def info(self):
    return self.details

In [ ]:
class Job(Document):
  def __init__(self, row):
    super().__init__(row)
    self.open_job_href = row["open_job-href"]
    self.job_title = row["job_title_simple"]
    self.company_name = row["company_name"]
    self.address = row["address"]
    self.salary_lb = row["salary_lb"]
    self.salary_hb = row["salary_hb"]
    self. job_desc = row["job_desc"]

  @property
  def job_text(self):
    return (
        f"""Job title: {self.job_title}
        Job link: {self.open_job_href}
        Company name: {self.company_name}
        Address: {self.address}
        Salary: {self.salary_lb} - {self.salary_hb}
        Job Description: {self.job_desc}
        """
        )


In [ ]:
jobs_arr = [Job(jobs.iloc[i]).job_text for i in range(jobs.shape[0])]
# print(len(jobs_arr))
jobs_arr[0]

'Job title: Administrative Assistant\n        Job link: https://www.mycareersfuture.gov.sg/job/admin/administrative-assistant-lyondell-south-asia-83e710469723cc60b524e8882ca1aed3?source=MCF&event=Search\n        Company name: LYONDELL SOUTH ASIA PTE LTD\n        Address: 78 SHENTON WAY 079120\n        Salary: 4400 - 4800\n        Job Description: Report discriminatory job ad to TAFEPRoles & ResponsibilitiesBasic Function\nProvide secretarial and administrative support to the Commercial team in a professional and efficient manner.  Also, responsible for overseeing general office duties for Singapore office.  \n\n\nRESPONSIBILITIES AND ACCOUNTABILITIES:\n\n  Preparation of monthly sales report data and comparison with budget and forecast.\n  Initialize communication with customers for receivable collection; handle business related queries as directed by Sales.\n  Well connected with various support functions and business group, able to channel customer / external requests to the right in

In [ ]:
job_dictionary = {f'id{i + 1}': Job(jobs.iloc[i]).job_text for i in range(jobs.shape[0])}
job_dictionary.get('id1')


'Job title: Administrative Assistant\n        Job link: https://www.mycareersfuture.gov.sg/job/admin/administrative-assistant-lyondell-south-asia-83e710469723cc60b524e8882ca1aed3?source=MCF&event=Search\n        Company name: LYONDELL SOUTH ASIA PTE LTD\n        Address: 78 SHENTON WAY 079120\n        Salary: 4400 - 4800\n        Job Description: Report discriminatory job ad to TAFEPRoles & ResponsibilitiesBasic Function\nProvide secretarial and administrative support to the Commercial team in a professional and efficient manner.  Also, responsible for overseeing general office duties for Singapore office.  \n\n\nRESPONSIBILITIES AND ACCOUNTABILITIES:\n\n  Preparation of monthly sales report data and comparison with budget and forecast.\n  Initialize communication with customers for receivable collection; handle business related queries as directed by Sales.\n  Well connected with various support functions and business group, able to channel customer / external requests to the right in

In [ ]:
!pip install ctransformers ctransformers[cuda]
import os
import torch
import torch.nn as nn
# import accelerate
from ctransformers import AutoModelForCausalLM

load_in_8bit = True
model_name_l = "TheBloke/OpenHermes-2.5-neural-chat-7B-v3-1-7B-GGUF"

# "TheBloke/OpenHermes-2.5-Mistral-7B-16k-GGUF", model_file="openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf", model_type="mistral")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 58.6 MB/s eta 0:00:00


In [ ]:
model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    model_file="openhermes-2.5-neural-chat-7b-v3-1-7b.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=100
    )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

(…)s-2.5-neural-chat-7b-v3-1-7b.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [ ]:
print(model_l("Can you provide information on the courses at SUTD for Systems Engineering?"))



At SUTD (Singapore University of Technology and Design), there is no specific degree program named "Systems Engineering". However, they offer several courses related to systems engineering under different majors or concentrations. For instance, under their School of Electrical & Computer Engineering, students can study systems and control through various elective modules like 'Digital Control Systems', 'Advanced Robotics' and 'Control System Design'. Additionally, the School of Engineering Product Development offers courses in design thinking, human-centered design, and product development which also contribute to systems engineering.

In summary, while there isn't a dedicated Systems Engineering degree program at SUTD, students can still explore related topics through various course offerings across different departments or disciplines.


In [ ]:
# load_in_8bit = True
# model_name_l = "lmsys/vicuna-7b-v1.5"

# model_l = AutoModelForCausalLM.from_pretrained(
#     model_name_l,
#     torch_dtype=torch.float16,
#     load_in_8bit=load_in_8bit,
#     device_map="auto"
# )
# tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)
# text = jobs.iloc[0]["company_name"]
# enc_text = tokenizer_l.encode(text)

# print(text,"; ", enc_text)

In [ ]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform(documents)

# Get the feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a dense array
dense_array = tfidf_matrix.toarray()

In [ ]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize

jobs_arr = [Job(jobs.iloc[i]).job_text for i in range(jobs.shape[0])]

tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate(jobs_arr[:10])]

# train the Doc2vec model
model = Doc2Vec(vector_size=768, min_count=2, epochs=50)

model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# get the document vectors
document_vectors = [model.infer_vector(word_tokenize(doc.lower())) for doc in jobs_arr[:10]]

In [ ]:
doc_v = np.array(document_vectors)
query_v = np.array()

In [ ]:
print(doc_v.shape)

(10, 768)


In [ ]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(jobs_arr[:3])]
max_epochs = 50
vec_size = 768
alpha = 0.025

# model = Doc2Vec(size=vec_size,alpha=alpha,min_alpha=0.00025,min_count=1,dm =1)
model = Doc2Vec(vector_size=vec_size, min_count=2, epochs=max_epochs)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=20
                )
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
Model Saved


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

model= Doc2Vec.load("d2v.model")

# print(jobs_arr[0].lower())
test_data = word_tokenize(jobs_arr[0].lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

q = word_tokenize("test query".lower())
q1 = model.infer_vector(test_data)
qq = q1.reshape(-1,1)
# print("V1_infer", v1)
cosine_similarity(v1.reshape(-1,1), qq)
# print(model.dv[jobs_arr[0].lower()])

V1_infer [-0.03639018 -0.27705067 -0.15294981  0.3804466   0.33455616  0.09786122
 -0.34581646 -0.13381538 -0.37978637  0.16735163  0.1045378   0.02033992
 -0.00371517 -0.2248033  -0.06395596 -0.25942817  0.03295412  0.46350947
 -0.39925906  0.10013211 -0.07690977 -0.0210587   0.10542077  0.06120914
  0.0464165  -0.4474234  -0.09305874 -0.3326058   0.00501572 -0.1824579
  0.14618799  0.37621582 -0.15469196 -0.12745336 -0.19247001  0.00700446
 -0.00364895 -0.13752472 -0.27112818 -0.00871265  0.02782659 -0.20580682
  0.39922124 -0.39551973  0.22193828 -0.20411724  0.07246928 -0.10566732
  0.2183701  -0.21600369 -0.1495397  -0.17228417 -0.18869463 -0.21378909
 -0.10920202  0.34958655  0.05987716 -0.07331405 -0.18637514  0.54520136
  0.06170122  0.3831774   0.3341433  -0.11612196  0.03193565 -0.01205495
  0.02106654 -0.00690208 -0.16805449 -0.21017009 -0.2498677  -0.04809769
 -0.5865547  -0.4073903  -0.3710754  -0.07810894 -0.14429948 -0.3920788
  0.24189955  0.04549432  0.18839303  0.2942

array([[ 1.,  1.,  1., ...,  1., -1., -1.],
       [ 1.,  1.,  1., ...,  1., -1., -1.],
       [ 1.,  1.,  1., ...,  1., -1., -1.],
       ...,
       [ 1.,  1.,  1., ...,  1., -1., -1.],
       [-1., -1., -1., ..., -1.,  1.,  1.],
       [-1., -1., -1., ..., -1.,  1.,  1.]], dtype=float32)

In [ ]:
v1.shape

(768,)

In [ ]:
v1

array([-0.03639018, -0.27705067, -0.15294981,  0.3804466 ,  0.33455616,
        0.09786122, -0.34581646, -0.13381538, -0.37978637,  0.16735163,
        0.1045378 ,  0.02033992, -0.00371517, -0.2248033 , -0.06395596,
       -0.25942817,  0.03295412,  0.46350947, -0.39925906,  0.10013211,
       -0.07690977, -0.0210587 ,  0.10542077,  0.06120914,  0.0464165 ,
       -0.4474234 , -0.09305874, -0.3326058 ,  0.00501572, -0.1824579 ,
        0.14618799,  0.37621582, -0.15469196, -0.12745336, -0.19247001,
        0.00700446, -0.00364895, -0.13752472, -0.27112818, -0.00871265,
        0.02782659, -0.20580682,  0.39922124, -0.39551973,  0.22193828,
       -0.20411724,  0.07246928, -0.10566732,  0.2183701 , -0.21600369,
       -0.1495397 , -0.17228417, -0.18869463, -0.21378909, -0.10920202,
        0.34958655,  0.05987716, -0.07331405, -0.18637514,  0.54520136,
        0.06170122,  0.3831774 ,  0.3341433 , -0.11612196,  0.03193565,
       -0.01205495,  0.02106654, -0.00690208, -0.16805449, -0.21

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.4/699.4 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "demo_docs"

client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [ ]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)
collection = client.create_collection(name=COLLECTION_NAME, embedding_function=embedding_func, metadata={"hnsw:space": "cosine"},)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
documents = jobs_arr[:3500]


collection.add(
    documents=documents,
    ids=[f"id{i}" for i in range(len(documents))],
)

In [ ]:
ques = "What are the courses related to Civil Engineering at NTU?"
print(model_l(ques))



Singapore University of Technology and Design (SUTD) offers a range of programs that are relevant to Systems Engineering. One such program is the Bachelor of Science in Engineering with a major in Systems Engineering, which focuses on developing students' skills in designing, analyzing, and managing complex systems. This program combines fundamental engineering principles with an interdisciplinary approach to prepare graduates for various industries.

The curriculum includes courses in areas like systems thinking, modeling and simulation, control theory, optimization, human-centered design, and project management. Students will also have the opportunity to specialize in a particular domain, such as cyber-physical systems, mechatronics, or healthcare engineering. Additionally, SUTD encourages students to participate in research projects, internships, and industry collaborations to gain practical experience and enhance their career prospects.

In summary, SUTD offers an extensive range

In [ ]:
sample_qna = pd.read_csv("drive/MyDrive/IR_Project/Sample qna.csv")
questions = list(sample_qna["Question"].values)
good_ans = list(sample_qna["Good"].values)

index = 1
questions = [questions[index]]
good_answer = [good_ans[index]]

In [ ]:
prediction = []

for q in questions:
  output = model_l(q)

In [ ]:
!pip install bert_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 997.2 kB/s eta 0:00:00


In [ ]:
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [ ]:
import evaluate

def getBERTScore(predictions, ground_truth):
  bertscore = evaluate.load("bertscore")
  bert_score = bertscore.compute(predictions=predictions, references=ground_truth, lang="en")
  print(f"BERTScore: {bert_score}")
  return bert_score

In [ ]:
bert_pred = [output]
current_acc = getBERTScore(bert_pred, good_answer)
print("BERT_score_before: ", current_acc)
print(f"BERT_score_before: {np.mean(current_acc['precision'])}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': [0.8523014783859253], 'recall': [0.8652108311653137], 'f1': [0.8587076663970947], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.2)'}
BERT_score_before:  {'precision': [0.8523014783859253], 'recall': [0.8652108311653137], 'f1': [0.8587076663970947], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.35.2)'}
BERT_score_before: 0.8523014783859253


In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer

load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

ImportError: ignored

In [ ]:
def response_gen(input_ids):
  temperature=0.7
  with torch.no_grad():
      generation_output = model_l.generate(
          input_ids=input_ids,
          temperature=temperature,
          top_p = 1.0,
          do_sample=True,
          return_dict_in_generate=True,
          max_new_tokens=300,
      )
  s = generation_output.sequences[0][len(input_ids[0]):]
  output = tokenizer_l.decode(s)
  return output

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def cos_sim(self, I):
  top_k_documents = self.wb[I]
  return cosine_similarity(top_k_documents, [self.xq])

In [ ]:
query_results = collection.query(
    query_texts=[ques],
    n_results=1,
)

query_results.keys()


query_results["documents"]



[["Einstein's theory of relativity revolutionized our understanding of space and time."]]

## Heuristic using vectors from vicuna tokenizer

In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.9 MB/s eta 0:00:00


In [ ]:
!pip install rank_bm25

In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
# from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, LlamaTokenizer
# from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class LinearModel(nn.Module):
    def __init__(self, dim_model=50, num_layers=2):
        super().__init__()
        self.num_layers = num_layers
        self.hidden = nn.Linear(dim_model, dim_model)
        self.classification_head = nn.Linear(dim_model, 1)

    def forward(self, src: torch.tensor) -> torch.tensor:
        for i in range(self.num_layers):
          src = self.hidden(src)
          src = torch.sigmoid(src)

        src = self.classification_head(src)
        src = torch.sigmoid(src)
        src = src.squeeze(-1)

        return src

In [ ]:
# Create class once
# Input into class: tokens from processing query through vicuna tokenizer
# Output: float 0<output<1 where 1 is closer to "job" and 0 is closer to "course"

class heuristic():
  def __init__(self):
    super().__init__()
    checkpoint_path = 'drive/MyDrive/IR_Project/heuristic_checkpoint.pt'
    if os.path.exists(checkpoint_path):
      self.model = LinearModel()
      checkpoint = torch.load(checkpoint_path)
      self.model.load_state_dict(checkpoint['model_state_dict'])
      self.model = self.model.to("cuda")
    else:
      raise ValueError("Cannot find checkpoint path")

  def process(self, label_features):
    the_feat = self.__process_tokenized(label_features)
    return self.model(the_feat).item()

  def __process_tokenized(self, label_features, max_tokens=50):
    label_feature = label_features["input_ids"].to("cuda")
    num_frames = label_feature.shape[1]
    if num_frames < max_tokens:
        padding = torch.zeros(label_feature.shape[0], max_tokens - num_frames).to("cuda")
        label_feature = torch.cat((label_feature, padding), dim=1)
    elif num_frames > max_tokens:
        label_feature = label_feature[:, :max_tokens]
    return label_feature

In [ ]:
# Testing
query = "Tell me more about engineering courses in SUTD"

tokenizer_l = LlamaTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5")
heuristical = heuristic()

tokenised_query = tokenizer_l(query, return_tensors='pt') # Just for testing, should be taken before passing the tokens into the LLM model
probability_of_token = heuristical.process(tokenised_query)
print(f"Probability of 'job': {probability_of_token}")

NameError: ignored

## Export vector to doc map to json file

In [ ]:
# from google.colab import files
# import json

# json_file_path = "vector_doc_map.json"

# vecdoc = {}
# for key, val in transformer.vector_doc_map.items():
#   k = str(list(key))
#   vecdoc[k] = val


# # Write the dictionary to the JSON file
# with open(json_file_path, 'w') as json_file:
#     json.dump(vecdoc, json_file)

# files.download('vector_doc_map.json')

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi


class FaissEncoder:
    def __init__(self, model_name='bert-base-nli-mean-tokens', dimension=768):
        self.model = SentenceTransformer(model_name)
        self.dimension = dimension
        self.index = None  # Index will be initialized during indexing step

    def index_strings(self, strings):
        # Encode strings into vectors
        vectors = [self.model.encode(s) for s in strings]
        vectors = np.array(vectors, dtype=np.float32)

        # Check if the dimension of the vectors matches the expected dimension
        if vectors.shape[1] != self.dimension:
            raise ValueError(f"Expected dimension {self.dimension}, but got {vectors.shape[1]}")

        # Initialize the Faiss index
        nlist = 100
        m = 8
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, self.dimension, nlist, m, 8)
        # Training the index with the vectors
        self.index.train(vectors)

        # Add the vectors to the index
        self.index.add(vectors)

    def encode_query(self, query):
        # Encode the query into a vector
        query_vector = self.model.encode(query)
        query_vector = np.array(query_vector, dtype=np.float32)

        # Check if the dimension of the query vector matches the expected dimension
        if query_vector.shape[0] != self.dimension:
            raise ValueError(f"Expected dimension {self.dimension}, but got {query_vector.shape[0]}")

        # Search for the nearest neighbors of the query in the Faiss index
        _, indices = self.index.search(query_vector.reshape(1, -1), 5)  # Searching for the top 5 nearest neighbors

        # Return the indices of the nearest neighbors
        return indices[0]





In [ ]:
# Example usage
query_string = "Test query string."

# Create and initialize the Faiss encoder with the lmsys/vicuna-7b-v1.5 model
encoder = FaissEncoder()

# Index the array of strings
encoder.index_strings(jobs_arr)

# Encode the query string and get the indices of the nearest neighbors
nearest_neighbor_indices = encoder.encode_query(query_string)

## can be used as ground_truth for finding n_bits
print("Indices of nearest neighbors:", nearest_neighbor_indices)


.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
encoder

# LSH Indexing and Cosine Similarity


In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# from rank_bm25 import BM25Okapi

class Transformer:
  def __init__(self, model_name='bert-base-nli-mean-tokens', dimension=768):
    self.model = SentenceTransformer(model_name)
    self.vector_doc_map = {}
    # self.vector_doc_map = loaded_vector_doc_map

    self.wb = []
    self.xq = None


  def doc_to_vector(self, jobs_arr):
    # create vector embeddings for the document
    vectors = [self.model.encode(s) for s in jobs_arr]
    vectors = np.array(vectors, dtype=np.float32)
    # create array to store
    self.wb = vectors

    # Create a dictionary mapping vectors to documents
    self.vector_doc_map = dict(zip(map(tuple, vectors), jobs_arr))

  def query_to_vector(self, query):
    xq = self.model.encode(query_string)
    self.xq = xq

  def lsh_index_search(self):
    d = self.wb.shape[1]
    nbits = 1000
    k =10

    # initialize the index using our vectors dimensionality (128) and nbits
    index = faiss.IndexLSH(d, nbits)
    # then add the data
    index.add(self.wb)
    xq0 = self.xq.reshape(1, -1)
    D, I = index.search(xq0, k=k)

    return I[0]

  def bm25_search(self, jobs_arr, query_string):

    tokenized_corpus = [doc.split(" ") for doc in jobs_arr]
    bm25 = BM25Okapi(tokenized_corpus)
    # create vector embedding for query
    tokenized_query = query_string.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)

    relevant_jobs = bm25.get_top_n(tokenized_query, jobs_arr, n=10)
    relevant_jobs_vectors = []
    for job in relevant_jobs:
      job_vector = next(key for key, value in self.vector_doc_map.items() if value == job)
      relevant_jobs_vectors.append(job_vector)

    return relevant_jobs_vectors

  def get_relevant_doc(self, I):
    relevant_documents = []

    # Retrieve the original document based on a vector
    for i in range(len(top_k_documents)):
        retrieved_document = self.vector_doc_map.get(tuple(top_k_documents[i]), "Document not found")
        print("\n================== Document", i, "====================\n")
        print(retrieved_document)

        relevant_documents.append(retrieved_document)
    return relevant_documents

  def cos_sim(self, I):
    top_k_documents = self.wb[I]
    return cosine_similarity(top_k_documents, [self.xq])


In [ ]:
# Example usage
ques = "What are the courses related to Civil Engineering at NTU?"


transformer = Transformer()
transformer.doc_to_vector(jobs_arr[:3500])
transformer.query_to_vector(ques)


In [ ]:
query_results = collection.query(
    query_texts=[ques],
    n_results=1,
)

query_results.keys()


# query_results["documents"]
relevant_docs = query_results["documents"][0]

# get vector of top n relevant jobs
relevant_docs_vectors = []
for doc in relevant_docs:
  for key, val in transformer.vector_doc_map.items():
    if val == doc:
      relevant_docs_vectors.append(key)
len(relevant_docs_vectors)

1

In [ ]:
transformer.cos_sim(relevant_docs_vectors, )

In [ ]:
transformer.wb.shape

(10, 768)

In [ ]:
lsh_I = transformer.lsh_index_search()
lsh_doc_sim = transformer.cos_sim(lsh_I)
lsh_rel_doc = transformer.get_relevant_doc(lsh_I)

print("\n================== LSH SEARCH ====================\n")
print("LSH similarity :" , lsh_doc_sim)
print("LSH relevant docs :" , lsh_rel_doc)

NameError: ignored

In [ ]:
bm25_I = transformer.bm25_search()
bm25_doc_sim = transformer.cos_sim(bm25_I)
bm25_rel_doc = transformer.get_relecant_doc(bm25_I)

print("\n================== BM25 SEARCH ====================\n")
print("BM25 similarity :" , lsh_doc_sim)
print("BM25 relevant docs :" , lsh_rel_doc)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model_name='bert-base-nli-mean-tokens'
model = SentenceTransformer(model_name)
vectors = [model.encode(s) for s in jobs_arr[:3]]
vectors = np.array(vectors, dtype=np.float32)

d = vectors.shape[1]
nbits = 1000

# initialize the index using our vectors dimensionality (128) and nbits
index = faiss.IndexLSH(d, nbits)
# then add the data
index.add(vectors)

# Example usage
query_string = "Test query string."

# create vector embedding for query
xq = model.encode(query_string)
xq0 = xq.reshape(1, -1)

D, I = index.search(xq0, k=10)
# the indexes of these vectors are returned to I

top_k_documents = vectors[I[0]]
cosine_similarity(top_k_documents, [xq])

ImportError: ignored

In [ ]:
D
len(top_k_documents)

10

In [ ]:
# Create a dictionary mapping vectors to documents
vector_to_document = dict(zip(map(tuple, vectors), jobs_arr))
relevant_documents = []



# Retrieve the original document based on a vector

query_vector = vectors[0]  # Replace with the vector you want to query
for i in range(len(top_k_documents)):
    retrieved_document = vector_to_document.get(tuple(top_k_documents[i]), "Document not found")
    print("\n================== Document", i, "====================\n")
    print(retrieved_document)

    relevant_documents.append(retrieved_document)

# print("Query Vector:", query_vector)
print("Retrieved Documents:", relevant_documents)
print(len(relevant_documents))


================== Document 0 ====================

Job title: Relief Infant Educarer
        Job link: https://www.mycareersfuture.gov.sg/job/education-training/relief-infant-educarer-cultivar-asia-451584551e136cf0fc9c8754bc917813?source=MCF&event=Search
        Company name: CULTIVAR ASIA PTE. LTD.
        Address: nan
        Salary: 2200 - 2800
        Job Description: Report discriminatory job ad to TAFEPRoles & ResponsibilitiesResponsibilities:

  Plan, prepare and execute curriculum
  Maintain learning materials and physical areas for the infants and toddlers
  Communicate with key stakeholders such as the parents, management, and co-workers
  Prepare periodic portfolio in accordance to regulatory frameworks
  Provide care and love with passion
  Ensure full compliance with ECDA requirements

Key requirements:

  Minimum ACEY, CIT, or HCIC
  Preferably registered with ECDA and has valid first aid certificate
  Preferably 1 year or more of infant care experience

Additional Info

# BM25 Elastic Search

In [ ]:
!pip install --upgrade langchain
!pip install elasticsearch

In [ ]:
!pip install rank_bm25

  Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB)


In [ ]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in jobs_arr]

bm25 = BM25Okapi(tokenized_corpus)
# Example usage
query_string = "Test query string."
# create vector embedding for query
xq = model.encode(query_string)

tokenized_query = query_string.split(" ")

doc_scores = bm25.get_scores(tokenized_query)

relevant_jobs = bm25.get_top_n(tokenized_query, jobs_arr, n=10)
print("jobs" ,relevant_jobs)

relevant_jobs_vectors = []
for job in relevant_jobs:
  job_vector = next(key for key, value in vector_to_document.items() if value == job)
  relevant_jobs_vectors.append(job_vector)

cosine_similarity(relevant_jobs_vectors, [xq])

NameError: ignored

In [ ]:
from elasticsearch import Elasticsearch
# from elasticsearch.common.exceptions import RequestError
# from elasticsearch.search.builder import SearchSourceBuilder

# Create an Elasticsearch client
es = Elasticsearch(

)

es.indices.create(index="my_index")

# Add documents to the index
for document in job_dictionary:
    es.index(index="my-index", id=document.key(), document=document.value())


# Create a search query
query = {"match": {query_string: query_string}}

# Create a search source builder
search_source_builder = SearchSourceBuilder()
search_source_builder.query(query)
search_source_builder.size(10)

# Add BM25 scoring to the search source builder
bm25_similarity = {
    "type": "BM25Similarity",
    "field": "content",
    "k1": 0.75,
    "b": 0.25
}
search_source_builder.ext(search_ext={"rescore": {"my_rescore": {"query": {"similarity": bm25_similarity}}}})

# Execute the search
search_result = es.search(index="my-index", body=search_source_builder.to_dict())

# Print the search results
for hit in search_result["hits"]["hits"]:
    print(hit["_source"])

client.search(index="my_index", query={
    "match": {
        "foo": "foo"
    }
})



ValueError: ignored

In [ ]:
k = 10
xq0 = xq.reshape(1, d)

while True:
    D, I = index.search(xq0, k=k)
    if D.any() != 0:
        print(k)
        print(D)
        break
    k += 10

10
[[320. 320. 322. 322. 324. 325. 326. 328. 328. 328.]]


In [ ]:
xq0 = xq.reshape(1, d)
k = 100

results = pd.DataFrame({
    'nbits': [],
    'cosine_sim': []
})

for epoch in range(1):
    print('.', end='')
    for nbits in [2, 4, 8, 16, 24, 32, 64, 128, 512, 1024]:
        index = faiss.IndexLSH(d, nbits)
        index.add(vectors)
        D, I = index.search(xq0, k=k)
        cos = cosine_similarity(vectors[I[0]], xq0)
        df = pd.DataFrame({
            'cosine_sim': cos.reshape(cos.shape[0])
        })
        df['nbits'] = nbits
        results = results.append(df, ignore_index=True)
        print(results)

.    nbits  cosine_sim
0     2.0    0.244977
1     2.0    0.318006
2     2.0    0.364592
3     2.0    0.277665
4     2.0    0.296219
..    ...         ...
95    2.0    0.395472
96    2.0    0.273684
97    2.0    0.392578
98    2.0    0.407721
99    2.0    0.336625

[100 rows x 2 columns]
     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
195    4.0    0.336625
196    4.0    0.357174
197    4.0    0.403380
198    4.0    0.253444
199    4.0    0.368497

[200 rows x 2 columns]
     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
295    8.0    0.386653
296    8.0    0.340924
297    8.0    0.344440
298    8.0    0.292808
299    8.0    0.310850

[300 rows x 2 columns]


<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)
<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)
<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)
<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)


     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
395   16.0    0.321790
396   16.0    0.329437
397   16.0    0.295356
398   16.0    0.316676
399   16.0    0.381279

[400 rows x 2 columns]
     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
495   24.0    0.358850
496   24.0    0.335492
497   24.0    0.364887
498   24.0    0.317982
499   24.0    0.373494

[500 rows x 2 columns]
     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
595   32.0    0.412376
596   32.0    0.429489
597   32.0    0.357217
598   32.0    0.403691
599   32.0    0.422879

[600 rows x 2 columns]


<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)
<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)
<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)


     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
695   64.0    0.361002
696   64.0    0.392316
697   64.0    0.376541
698   64.0    0.446427
699   64.0    0.382903

[700 rows x 2 columns]
     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
795  128.0    0.363909
796  128.0    0.374496
797  128.0    0.431760
798  128.0    0.415871
799  128.0    0.422294

[800 rows x 2 columns]


<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)
<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)


     nbits  cosine_sim
0      2.0    0.244977
1      2.0    0.318006
2      2.0    0.364592
3      2.0    0.277665
4      2.0    0.296219
..     ...         ...
895  512.0    0.377289
896  512.0    0.440817
897  512.0    0.445550
898  512.0    0.383844
899  512.0    0.457962

[900 rows x 2 columns]
      nbits  cosine_sim
0       2.0    0.244977
1       2.0    0.318006
2       2.0    0.364592
3       2.0    0.277665
4       2.0    0.296219
..      ...         ...
995  1024.0    0.420870
996  1024.0    0.421934
997  1024.0    0.443157
998  1024.0    0.448775
999  1024.0    0.420217

[1000 rows x 2 columns]


<ipython-input-16-a0672c3c1de8>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(df, ignore_index=True)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.lineplot(data=results, x='nbits', y='cosine_sim')

In [ ]:
df.to_csv('your_file_name.csv', index=False)
